## Di sini masih pake cara manual krn sy ga tau lol aph itu interpolasi

In [19]:
import pandas as pd

In [20]:
df_train_nan = pd.read_csv('./dataset/train_nan.csv')
df_train_nan['datetime_iso'] = pd.to_datetime(df_train_nan['datetime_iso'])

In [21]:
df_train_median = pd.read_csv('./dataset/train_median.csv')
df_train_median['datetime_iso'] = pd.to_datetime(df_train_median['datetime_iso'])

In [22]:
df_train_nan = df_train_nan.drop(['visibility','sea_level','grnd_level','rain_3h','snow_1h','snow_3h'],axis = 1)
df_train_median = df_train_median.drop(['visibility','sea_level','grnd_level','rain_3h','snow_1h','snow_3h'],axis = 1)


In [23]:
null_cols = df_train_nan.columns[df_train_nan.isna().any()].tolist()
null_cols_2 = df_train_median.columns[df_train_nan.isna().any()].tolist()

In [24]:
df_train_nan.isna().sum()

datetime            0
datetime_iso        0
time-zone           0
temp             5557
d_point          9441
feels            5128
min_temp         5219
max_temp         5854
prssr            9888
hum              9549
wind_spd        16317
wind_deg         4235
rain_1h         52957
clouds          40927
dtype: int64

In [25]:
df_train_median.isna().sum()

datetime            0
datetime_iso        0
time-zone           0
temp                0
d_point             0
feels               0
min_temp            0
max_temp            0
prssr               0
hum                 0
wind_spd        16317
wind_deg            0
rain_1h             0
clouds              0
dtype: int64

## Fungsi di sini

In [26]:
df_train_nan['date'] = df_train_nan['datetime_iso'].dt.date
df_train_median['date'] = df_train_median['datetime_iso'].dt.date

In [27]:
def filling_missing_values(df, col: str):
  # manual filling
  df[col] = df[col].fillna(df[col].shift() + df[col].shift(-1) / 2)  
  group_by_date = round(df.groupby('date')['wind_spd'].mean(), 2)
  df[col] = df[col].fillna(df['date'].map(group_by_date))

def filling_missing_values_with_interpolation(df):
  # time interpolation, nek pake ini yang bawah di-comment
  df = df.set_index('datetime_iso') 
  df = df.interpolate(method='time')

  # selain itu
  df = df.interpolate(method='backfill')


In [29]:
df_list = [df_train_nan, df_train_median]

for col in null_cols:
  for item in df_list:
    # ganti di sini adik2
    filling_missing_values(item, col)
    print(item.isna().sum())
  

datetime            0
datetime_iso        0
time-zone           0
temp                0
d_point          9441
feels            5128
min_temp         5219
max_temp         5854
prssr            9888
hum              9549
wind_spd        16317
wind_deg         4235
rain_1h         52957
clouds          40927
date                0
dtype: int64
datetime            0
datetime_iso        0
time-zone           0
temp                0
d_point             0
feels               0
min_temp            0
max_temp            0
prssr               0
hum                 0
wind_spd        16317
wind_deg            0
rain_1h             0
clouds              0
date                0
dtype: int64
datetime            0
datetime_iso        0
time-zone           0
temp                0
d_point             0
feels            5128
min_temp         5219
max_temp         5854
prssr            9888
hum              9549
wind_spd        16317
wind_deg         4235
rain_1h         52957
clouds          40927
date  

In [ ]:

# drop helper column
df_train_nan = df_train_nan.drop('date', axis=1)
df_train_median = df_train_median.drop('date', axis=1)

In [ ]:
df_train_nan.to_csv('./dataset/train_nan_cleaned.csv', index=False)
df_train_median.to_csv('./dataset/train_nan_cleaned.csv', index=False)

## Di bawah ini ga kepake xD

In [ ]:
index = df_train_nan[df_train_nan['wind_spd'].isna()].index

# liat data yang missing sama intervalnya
missing_with_intervals = df_train_nan[(df_train_nan.index.isin(index)) | (df_train_nan.index.isin(index-1)) | (df_train_nan.index.isin(index+1))]
missing_with_intervals.head(12)


,datetime,datetime_iso,time-zone,temp,d_point,feels,min_temp,max_temp,prssr,hum,wind_spd,wind_deg,rain_1h,clouds


In [ ]:
# isi missing value dengan nilai rata-rata dari data sebelum dan sesudahnya
df_train_nan['wind_spd'] = df_train_nan['wind_spd'].fillna((df_train_nan['wind_spd'].shift() + df_train_nan['wind_spd'].shift(-1))/2)

In [ ]:
# cek lagi missing value
df_train_nan.iloc[15:22]

,datetime,datetime_iso,time-zone,temp,d_point,feels,min_temp,max_temp,prssr,hum,wind_spd,wind_deg,rain_1h,clouds
15,284050800,1979-01-01 15:00:00+00:00,28800,24.63,24.46,25.73,23.80,25.36,1012.0,99.0,0.90,301.0,0.0,100.0
16,284054400,1979-01-01 16:00:00+00:00,28800,24.60,24.43,25.70,23.69,25.39,1012.0,148.5,0.92,291.0,0.0,100.0
17,284058000,1979-01-01 17:00:00+00:00,28800,24.36,24.19,25.44,23.43,25.12,1012.0,99.0,0.80,296.0,0.0,100.0
18,284061600,1979-01-01 18:00:00+00:00,28800,24.09,23.92,25.14,23.18,24.87,1011.0,99.0,0.66,311.0,0.0,100.0
19,284065200,1979-01-01 19:00:00+00:00,28800,23.95,23.78,24.99,22.94,24.75,1010.0,99.0,0.77,281.0,0.0,100.0
20,284068800,1979-01-01 20:00:00+00:00,28800,23.82,23.65,24.84,22.77,24.64,1011.0,99.0,1.23,267.0,0.0,100.0
21,284072400,1979-01-01 21:00:00+00:00,28800,23.48,23.48,24.49,22.52,24.30,1011.0,100.0,0.92,289.0,0.0,98.0


In [ ]:
df_train_nan.isna().sum()

datetime        0
datetime_iso    0
time-zone       0
temp            0
d_point         0
feels           0
min_temp        0
max_temp        0
prssr           0
hum             0
wind_spd        0
wind_deg        0
rain_1h         0
clouds          0
dtype: int64

masih ada karena ya bisa aja samping kanan kiri belio isinya NaN :D

In [ ]:
index_layer2 = df_train_nan[df_train_nan['wind_spd'].isna()].index

# ambil satu ajah ckup
start_interval = index_layer2[0] - 4
end_interval = index_layer2[0] + 4

df_train_nan.iloc[start_interval:end_interval]

IndexError: index 0 is out of bounds for axis 0 with size 0

betyul sebelahnya NaN, sbenere bisa aja ambil kirinya tapi kalo NaN lagi gimana? ambil kirinya kagi? capek ga luwh

In [ ]:
df_train_nan['date'] = df_train_nan['datetime_iso'].dt.date

# jd gweh ngegroup by date then cari mean-nyah
group_by_date = round(df_train_nan.groupby('date')['wind_spd'].mean(), 2)

In [ ]:
group_by_date.head(5)

date
1979-01-01    0.81
1979-01-02    0.71
1979-01-03    0.84
1979-01-04    0.82
1979-01-05    1.04
Name: wind_spd, dtype: float64

In [ ]:
# isi missing value dengan rata2 di dalam group_by_date
df_train_nan['wind_spd'] = df_train_nan['wind_spd'].fillna(df_train_nan['date'].map(group_by_date))

In [ ]:
df_train_nan.iloc[start_interval:end_interval]

,datetime,datetime_iso,time-zone,temp,d_point,feels,min_temp,max_temp,prssr,hum,wind_spd,wind_deg,rain_1h,clouds,date
2264,292147200,1979-04-05 08:00:00+00:00,28800,27.75,26.14,33.12,27.08,28.35,1006.0,91.0,0.64,333.0,0.11,72.0,1979-04-05
2265,292150800,1979-04-05 09:00:00+00:00,28800,26.46,25.59,26.46,25.94,29.02,1006.0,95.0,0.47,325.0,0.11,57.0,1979-04-05
2266,292154400,1979-04-05 10:00:00+00:00,28800,26.62,25.39,26.62,26.17,27.48,1007.0,93.0,1.03,54.0,1.24,70.0,1979-04-05
2267,292158000,1979-04-05 11:00:00+00:00,28800,26.03,24.81,26.03,25.64,26.58,1008.0,93.0,1.08,8.0,0.21,84.0,1979-04-05
2268,292161600,1979-04-05 12:00:00+00:00,28800,25.59,24.90,26.71,24.92,26.20,1009.0,96.0,1.08,11.0,0.00,98.0,1979-04-05
2269,292165200,1979-04-05 13:00:00+00:00,28800,25.45,24.76,26.56,24.85,26.07,1009.0,96.0,1.08,349.0,0.00,100.0,1979-04-05
2270,292168800,1979-04-05 14:00:00+00:00,28800,25.62,25.11,26.77,24.84,26.36,1010.0,97.0,0.97,348.0,0.00,100.0,1979-04-05
2271,292172400,1979-04-05 15:00:00+00:00,28800,24.91,24.74,26.04,24.24,25.78,1009.0,99.0,1.30,342.0,0.00,98.0,1979-04-05
